In [ ]:
# ✅ MAIN.PY
import pickle
from inges import DataIngestion
from Trainer import ModelTrainer
from Validator import ModelEvaluation
from Logger import Logger
import mlflow

if __name__ == "__main__":
    log = Logger()
    try:
        mlflow.set_tracking_uri("http://127.0.0.1:5000")
        mlflow.set_experiment("Marital Status Prediction")

        with mlflow.start_run(run_name="Main Pipeline"):
            log.info("Pipeline started")

            # Data Ingestion
            data = DataIngestion(file_path="D:/your_path/World_Marriage_Dataset(preprocessed)(oversampling)(with_randomness).csv")
            X_train, X_test, y_train, y_test = data.load_data()
            log.info("Data ingestion done")

            # Model Training
            trainer = ModelTrainer(X_train, y_train)
            model = trainer.train_model()
            log.info("Model training done")

            # Save model
            with open("DS_Classifier.pkl", "wb") as f:
                pickle.dump(model, f)
            mlflow.log_artifact("DS_Classifier.pkl")

            # Evaluation
            evaluator = ModelEvaluation()
            metrics = evaluator.evaluate_model(model, X_test, y_test)
            log.info(f"Evaluation Done: {metrics}")

            mlflow.set_tag("pipeline_status", "SUCCESS")

    except Exception as e:
        log.error(str(e))
        while mlflow.active_run():
            mlflow.end_run(status="FAILED")
        raise
